# 구글에서 이미지를 크롤링 합니다

# 라이브러리 설치

In [1]:
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install webdriver-manager

# 중요!! 크롬 버전을 설정하세요

In [2]:
CHROME_VERSION = '90.0.4430.24'

# 중요!! 다운로드 폴더를 적어주세요
{다운로드폴더}/{검색어}/ 에 이미지가 저장됩니다

In [3]:
import os
DOWNLOAD_DIR = './images/google'
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# 코드 시작

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import urllib.request
from pathlib import Path

SCROLL_PAUSE_TIME = 1
BASE_URL ="https://www.google.com/search?hl=ko&tbm=isch&q="

In [5]:
def crawl_image(query):
    image_dir = os.path.join(DOWNLOAD_DIR, query)
    if not os.path.exists(image_dir):
        os.makedirs(image_dir, exist_ok=True)
    
    driver = webdriver.Chrome(ChromeDriverManager(version=CHROME_VERSION).install())
    driver.get(BASE_URL)
    elem = driver.find_element_by_name("q")
    elem.send_keys(query)
    elem.send_keys(Keys.RETURN)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(SCROLL_PAUSE_TIME)

        continue_button = driver.find_element_by_xpath('//input[@class="mye4qd"]')
        if continue_button.is_displayed():
            continue_button.click()
            continue
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    prev_url = None

    images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
    count = 1
    for image in images:
        try: 
            image.click()
            time.sleep(2)
            img_url = driver.find_element_by_xpath('//div[@class="tvh9oe BIB1wf"]/c-wiz/div/div/div/div/div[2]/a/img').get_attribute('src')
            if img_url.startswith('http') and (img_url != prev_url):
                file_name = Path(img_url).name
                urllib.request.urlretrieve(img_url, os.path.join(image_dir, file_name))
                prev_url = img_url
                count = count + 1

        except:
            pass
            
    driver.close()

# 크롤링은 아래에서

In [8]:
crawl_image('DDP')



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Driver [/home/myungjin-kim/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache
